In [1]:
# Imports
import pandas as pd
import numpy as np
import datetime
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error

# Definitions
pd.set_option('display.float_format', lambda x: '%.3f' % x)
%matplotlib inline
FOLDS = 5
N_JOBS = 6
SEED = 2016


In [2]:
# Load test sets
y_test = pd.read_csv("clean_data/y_test.csv").TARGET
test = pd.read_csv("clean_data/test_Base.csv")
print("y_test : " + str(y_test.shape))
print("test : " + str(test.shape))

y_test : (156123,)
test : (300891, 34)


In [3]:
# Load  predictions on held out fold
XGB_Time = np.array(pd.read_csv("y_test_preds/XGB_Time_0-01_goodSettings.csv").TARGET)
LGBM_Time = np.array(pd.read_csv("y_test_preds/LGBM_Time.csv").TARGET)
'''XGB_Base = np.array(pd.read_csv("y_test_preds/XGB_Base.csv").TARGET)
XGB_Time = np.array(pd.read_csv("y_test_preds/XGB_Time.csv").TARGET)
LightGBM_Base = np.array(pd.read_csv("y_test_preds/LightGBM_Base.csv").TARGET)
LightGBM_Time = np.array(pd.read_csv("y_test_preds/LightGBM_Time.csv").TARGET)
RandomForests_Base = np.array(pd.read_csv("y_test_preds/RandomForests_Base.csv").TARGET)
RandomForests_Time = np.array(pd.read_csv("y_test_preds/RandomForests_Time.csv").TARGET)
ExtraTrees_Base = np.array(pd.read_csv("y_test_preds/ExtraTrees_Base.csv").TARGET)
ExtraTrees_Time = np.array(pd.read_csv("y_test_preds/ExtraTrees_Time.csv").TARGET)
Lasso_Base = np.array(pd.read_csv("y_test_preds/Lasso_Base.csv").TARGET)
Lasso_Time = np.array(pd.read_csv("y_test_preds/Lasso_Time.csv").TARGET)'''

preds_y_test = []
preds_y_test.append(XGB_Time)
preds_y_test.append(LGBM_Time)
'''preds_y_test.append(XGB_Base)
preds_y_test.append(XGB_Time)
preds_y_test.append(LightGBM_Base)
preds_y_test.append(LightGBM_Time)
preds_y_test.append(RandomForests_Base)
preds_y_test.append(RandomForests_Time)
preds_y_test.append(ExtraTrees_Base)
preds_y_test.append(ExtraTrees_Time)
preds_y_test.append(Lasso_Base)
preds_y_test.append(Lasso_Time)'''

names = []
names.append("XGB_Time")
names.append("LGBM_Time")
'''names.append("XGB_Base")
names.append("XGB_Time")
names.append("LightGBM_Base")
names.append("LightGBM_Time")
names.append("RandomForests_Base")
names.append("RandomForests_Time")
names.append("ExtraTrees_Base")
names.append("ExtraTrees_Time")
names.append("Lasso_Base")
names.append("Lasso_Time")'''

print(XGB_Time.shape)
print(LGBM_Time.shape)
'''print(XGB_Base.shape)
print(XGB_Time.shape)
print(LightGBM_Base.shape)
print(LightGBM_Time.shape)
print(RandomForests_Base.shape)
print(RandomForests_Time.shape)'''


(156123,)
(156123,)


'print(XGB_Base.shape)\nprint(XGB_Time.shape)\nprint(LightGBM_Base.shape)\nprint(LightGBM_Time.shape)\nprint(RandomForests_Base.shape)\nprint(RandomForests_Time.shape)'

In [4]:
# Remind scores of single models on held out fold
print("XGB_Time on held out fold : " + str(mean_squared_error(y_test, XGB_Time)))
print("LGBM_Time on held out fold : " + str(mean_squared_error(y_test, LGBM_Time)))
'''print("XGB_Base on held out fold : " + str(mean_squared_error(y_test, XGB_Base)))
print("XGB_Time on held out fold : " + str(mean_squared_error(y_test, XGB_Time)))
print("LightGBM_Base on held out fold : " + str(mean_squared_error(y_test, LightGBM_Base)))
print("LightGBM_Time on held out fold : " + str(mean_squared_error(y_test, LightGBM_Time)))
print("RandomForests_Base on held out fold : " + str(mean_squared_error(y_test, RandomForests_Base)))
print("RandomForests_Time on held out fold : " + str(mean_squared_error(y_test, RandomForests_Time)))
print("ExtraTrees_Base on held out fold : " + str(mean_squared_error(y_test, ExtraTrees_Base)))
print("ExtraTrees_Time on held out fold : " + str(mean_squared_error(y_test, ExtraTrees_Time)))
print("Lasso_Base on held out fold : " + str(mean_squared_error(y_test, Lasso_Base)))
print("Lasso_Time on held out fold : " + str(mean_squared_error(y_test, Lasso_Time)))'''


XGB_Time on held out fold : 97.5314318575
LGBM_Time on held out fold : 110.296867735


'print("XGB_Base on held out fold : " + str(mean_squared_error(y_test, XGB_Base)))\nprint("XGB_Time on held out fold : " + str(mean_squared_error(y_test, XGB_Time)))\nprint("LightGBM_Base on held out fold : " + str(mean_squared_error(y_test, LightGBM_Base)))\nprint("LightGBM_Time on held out fold : " + str(mean_squared_error(y_test, LightGBM_Time)))\nprint("RandomForests_Base on held out fold : " + str(mean_squared_error(y_test, RandomForests_Base)))\nprint("RandomForests_Time on held out fold : " + str(mean_squared_error(y_test, RandomForests_Time)))\nprint("ExtraTrees_Base on held out fold : " + str(mean_squared_error(y_test, ExtraTrees_Base)))\nprint("ExtraTrees_Time on held out fold : " + str(mean_squared_error(y_test, ExtraTrees_Time)))\nprint("Lasso_Base on held out fold : " + str(mean_squared_error(y_test, Lasso_Base)))\nprint("Lasso_Time on held out fold : " + str(mean_squared_error(y_test, Lasso_Time)))'

In [5]:
# Optimize ensemble weights on held out fold

# Define function to minimize
def mse_target_func(weights):
    final_prediction = 0
    for weight, prediction in zip(weights, preds_y_test):
        final_prediction += weight * prediction
    return (mean_squared_error(y_test, final_prediction))

scores = []
weights = []
for i in range(100):
    # Choose many random starting weights
    starting_values = np.random.uniform(size = len(preds_y_test))
    
    # Our weights are bound between 0 and 1
    bounds = [(0, 1)] * len(preds_y_test)

    res = minimize(
        mse_target_func, 
        starting_values, 
        method = "SLSQP", 
        bounds = bounds, 
        options = {"maxiter" : 10000})
    
    scores.append(res["fun"])
    weights.append(res["x"])

bestSC = np.min(scores)
bestWght = weights[np.argmin(scores)]

print("\n Ensemble Score: {}".format(bestSC))
print("\n Best Weights: {}".format(bestWght))
print("\n Names: {}".format(names))


 Ensemble Score: 94.37539762164165

 Best Weights: [ 0.91734011  0.00941541]

 Names: ['XGB_Time', 'LGBM_Time']


In [23]:
# Load predictions on whole test set
preds_test = []
preds_test.append(pd.read_csv("test_preds/XGB_Time_0-01_goodSettings.csv", sep = ";"))
preds_test.append(pd.read_csv("test_preds/LGBM_Time.csv", sep = ";"))
'''preds_test.append(pd.read_csv("test_preds/XGB_Base.csv"))
preds_test.append(pd.read_csv("test_preds/XGB_Time.csv"))
preds_test.append(pd.read_csv("test_preds/LightGBM_Base.csv"))
preds_test.append(pd.read_csv("test_preds/LightGBM_Time.csv"))
preds_test.append(pd.read_csv("test_preds/RandomForests_Base.csv"))
preds_test.append(pd.read_csv("test_preds/RandomForests_Time.csv"))
preds_test.append(pd.read_csv("test_preds/ExtraTrees_Base.csv"))
preds_test.append(pd.read_csv("test_preds/ExtraTrees_Time.csv"))
preds_test.append(pd.read_csv("test_preds/Lasso_Base.csv"))
preds_test.append(pd.read_csv("test_preds/Lasso_Time.csv"))'''


'preds_test.append(pd.read_csv("test_preds/XGB_Base.csv"))\npreds_test.append(pd.read_csv("test_preds/XGB_Time.csv"))\npreds_test.append(pd.read_csv("test_preds/LightGBM_Base.csv"))\npreds_test.append(pd.read_csv("test_preds/LightGBM_Time.csv"))\npreds_test.append(pd.read_csv("test_preds/RandomForests_Base.csv"))\npreds_test.append(pd.read_csv("test_preds/RandomForests_Time.csv"))\npreds_test.append(pd.read_csv("test_preds/ExtraTrees_Base.csv"))\npreds_test.append(pd.read_csv("test_preds/ExtraTrees_Time.csv"))\npreds_test.append(pd.read_csv("test_preds/Lasso_Base.csv"))\npreds_test.append(pd.read_csv("test_preds/Lasso_Time.csv"))'

In [26]:
# Prepare submission using optimal weights
preds_ens = 0
for i in range(0, len(bestWght)) :
    preds_ens += (bestWght[i] * preds_test[i])

In [27]:
# Save test set preds on file
now = datetime.datetime.now()
name = "test_preds/preds_ens_" + str(now.strftime("%Y-%m-%d-%H-%M")) + ".csv"
pd.DataFrame({"ID": test.ID, "TARGET": preds_ens.TARGET}, columns = ["ID", "TARGET"]).to_csv(name, index = False)